# Using Data Analysis Techniques and Machine Learning to Search for Dark Matter (IN PROGRESS)

## Introduction, etc. here

In [1]:
import pandas as pd #So we can store data in DataFrames (an object type that makes for easier analysis)
import numpy as np #used for calculations
import uproot3 #allows us to accesss the data within the files we are using
import matplotlib.pyplot as plt #used for plotting graphs


from matplotlib.lines import Line2D #To put a dashed line in the legend (needed for this notebook?)
from matplotlib.ticker import AutoMinorLocator,MaxNLocator # for minor ticks and forcing integer tick labels (needed?)

In [2]:
data_path = 'C:\\Users\\Chris\\Desktop\\Internship\\Data\\mc_306109.dmV_Zll_MET40_DM1_MM800.exactly2lep.root'
#notice this is in the .root format used for CERN data, so we need the uproot3 module to access it

Data from: https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/exactly2lep/MC/

In [3]:
file_open = uproot3.open(data_path)["mini"] #open data file (whats mini?)

In [4]:
uproot3.numentries(data_path, "mini") #check the number of rows of data

5206

In [5]:
keys = file_open.keys()
show = file_open.show()
#these two are similar and show the labels of all the columns of data

runNumber                  (no streamer)              asdtype('>i4')
eventNumber                (no streamer)              asdtype('>i4')
channelNumber              (no streamer)              asdtype('>i4')
mcWeight                   (no streamer)              asdtype('>f4')
scaleFactor_PILEUP         (no streamer)              asdtype('>f4')
scaleFactor_ELE            (no streamer)              asdtype('>f4')
scaleFactor_MUON           (no streamer)              asdtype('>f4')
scaleFactor_PHOTON         (no streamer)              asdtype('>f4')
scaleFactor_TAU            (no streamer)              asdtype('>f4')
scaleFactor_BTAG           (no streamer)              asdtype('>f4')
scaleFactor_LepTRIGGER     (no streamer)              asdtype('>f4')
scaleFactor_PhotonTRIGGER  (no streamer)              asdtype('>f4')
trigE                      (no streamer)              asdtype('bool')
trigM                      (no streamer)              asdtype('bool')
trigP                      (no s

In [6]:
len(file_open.array(keys[69])) #the number of entries/rows in one of the arrays in the data

5206

In [7]:
keys[68] #gives the name of one of the arrays/columns in the data

b'XSection'

In [8]:
file_open.array(keys[68]) #viewing the data in one of the arrays/columns in the dataset

array([0.005016, 0.005016, 0.005016, ..., 0.005016, 0.005016, 0.005016],
      dtype=float32)

In [9]:
data_dict = {} #dictionary to hold all the data that we will be looking at


relevant_data = ['runNumber','eventNumber','channelNumber','mcWeight','lep_n','lep_truthMatched','lep_trigMatched',
                'lep_pt','lep_eta','lep_phi','lep_E','lep_z0','lep_charge','lep_type','lep_isTightID','lep_ptcone30',
                 'lep_etcone20','lep_trackd0pvunbiased','lep_tracksigd0pvunbiased','met_et','lep_pt_syst','XSection',
                 'SumWeights']
# these are the headings of the columns of data that we will be looking at
relevant_data_locations = [0,1,2,3,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,63,68,69]


#the below lines read the data that we are going to use into the dictionary above
counter = 0
for i in relevant_data_locations:
    data_dict[relevant_data[counter]] = file_open.array(keys[i]) 
    #creates a new key in the dictionary using the entries in relevant_data, and adds the corresponding data values by opening
    #the right array of data (using the column number given in relevant_data_locations)
    counter+=1
    

#data_dict
#uncomment above to view the dictionary

In [10]:
raw_data_df = pd.DataFrame(data_dict) #makes a DataFrame from the constructed dictionary
raw_data_df #view the dataframe

,runNumber,eventNumber,channelNumber,mcWeight,lep_n,lep_truthMatched,lep_trigMatched,lep_pt,lep_eta,lep_phi,...,lep_type,lep_isTightID,lep_ptcone30,lep_etcone20,lep_trackd0pvunbiased,lep_tracksigd0pvunbiased,met_et,lep_pt_syst,XSection,SumWeights
0,284500,46,306109,1.0,2,"[True, False]","[True, False]","[100990.07, 7426.121]","[-0.7682709, 0.72368354]","[-1.0182151, -2.1507058]",...,"[11, 11]","[True, True]","[0.0, 0.0]","[1215.9817, -544.9584]","[-0.0099716205, 0.009749278]","[1.1405811, 0.6889102]",348277.750000,"[668.81635, 66.78632]",0.005016,10000.0
1,284500,205,306109,1.0,2,"[True, True]","[True, True]","[93777.734, 31697.932]","[-0.5999321, -0.89178747]","[0.17214498, -1.7731516]",...,"[13, 13]","[True, True]","[0.0, 0.0]","[1523.3855, 1132.3503]","[-0.0024083678, -0.0005763579]","[0.32693756, 0.06280022]",104342.265625,"[2037.9049, 107.43644]",0.005016,10000.0
2,284500,235,306109,1.0,2,"[True, True]","[True, True]","[84696.75, 68743.09]","[1.1828485, 1.098033]","[0.37650913, -0.83802056]",...,"[11, 11]","[True, True]","[0.0, 0.0]","[3794.2654, -442.27618]","[0.008800238, 0.0039059948]","[1.0115662, 0.46925277]",128276.781250,"[428.07352, 308.5227]",0.005016,10000.0
3,284500,482,306109,1.0,2,"[True, True]","[True, False]","[95645.26, 22749.984]","[0.03707598, -0.2220163]","[-0.08936702, 2.6688776]",...,"[11, 11]","[True, True]","[0.0, 0.0]","[1735.391, -518.5956]","[-0.013859223, 0.019677153]","[1.790706, 1.7911686]",63393.468750,"[1129.4507, 80.28829]",0.005016,10000.0
4,284500,630,306109,1.0,2,"[True, True]","[True, False]","[128806.86, 15045.653]","[-0.34280708, -0.80403394]","[-2.7522888, -0.12069551]",...,"[11, 11]","[True, True]","[0.0, 0.0]","[-272.75995, -339.5269]","[-0.020453207, 0.01955478]","[3.6392374, 1.2386608]",123417.039062,"[497.85922, 170.0829]",0.005016,10000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5201,284500,9440,306109,1.0,2,"[True, True]","[True, True]","[72181.87, 51158.71]","[0.5563412, -0.18103471]","[3.0907967, 1.6497777]",...,"[13, 13]","[True, True]","[0.0, 0.0]","[-761.564, -761.564]","[0.011878228, -0.0021623962]","[1.959224, 0.33713147]",106641.679688,"[706.1843, 566.79584]",0.005016,10000.0
5202,284500,9558,306109,1.0,2,"[True, True]","[True, False]","[306378.8, 21968.338]","[0.6307145, 0.52217853]","[-2.6945016, -1.5103921]",...,"[13, 13]","[True, True]","[0.0, 0.0]","[814.3611, -943.9758]","[-0.006906736, 0.007331518]","[1.3408303, 0.7073485]",347592.312500,"[37542.293, 155.38966]",0.005016,10000.0
5203,284500,9697,306109,1.0,2,"[True, True]","[True, True]","[70584.914, 54114.15]","[-1.3256004, -1.0071476]","[-1.9375601, 3.0470493]",...,"[13, 13]","[True, True]","[0.0, 0.0]","[-389.3839, -726.9248]","[0.011187258, -0.024736652]","[1.4650736, 2.6470094]",110196.546875,"[683.6778, 116.885124]",0.005016,10000.0
5204,284500,9922,306109,1.0,2,"[True, True]","[True, True]","[102851.836, 31841.15]","[1.6040598, 1.010763]","[0.39593637, 2.0243583]",...,"[11, 11]","[True, True]","[1450.1215, 0.0]","[1760.7639, -980.12756]","[0.014684265, 0.0007587629]","[1.0974138, 0.045595814]",97570.312500,"[1874.4982, 65.11265]",0.005016,10000.0


In [11]:
raw_data_df['lep_pt'] #view one column of the dataframe

0        [100990.07, 7426.121]
1       [93777.734, 31697.932]
2         [84696.75, 68743.09]
3        [95645.26, 22749.984]
4       [128806.86, 15045.653]
                 ...          
5201      [72181.87, 51158.71]
5202     [306378.8, 21968.338]
5203     [70584.914, 54114.15]
5204    [102851.836, 31841.15]
5205     [50489.36, 35715.906]
Name: lep_pt, Length: 5206, dtype: object

In [12]:
len(relevant_data)

23

In the dataframe we see many columns that appear to only one unique value, these will not need to be histogrammed.

First we will check the number of unique values in each column of the Dataframe

In [13]:
checking = [1,1,1,2,3,3,3,3,3,3,3,3,3,4,5,5]
set(checking)

{1, 2, 3, 4, 5}

In [39]:
for i in raw_data_df:
    array_comparison=[] #list to hold bool values comparing arrays to check wheteher they should be histogrammed
    array_similarity =[]
    if type(raw_data_df[i][0])==np.ndarray:
        print('LIST HERE')
        for j in raw_data_df[i]:
            if (raw_data_df[i][0]==raw_data_df['lep_truthMatched'][1]).all():
            
        
    #print(i, len(set(raw_data_df[i])))
    #print(i,raw_data_df[i])
    

LIST HERE
LIST HERE
LIST HERE
LIST HERE
LIST HERE
LIST HERE
LIST HERE
LIST HERE
LIST HERE
LIST HERE
LIST HERE
LIST HERE
LIST HERE
LIST HERE
LIST HERE


In [52]:
#type()
for i in range(len(raw_data_df['lep_truthMatched'])):
    if (raw_data_df['lep_truthMatched'][0]==raw_data_df['lep_truthMatched'][i]).all():
        print('same')
#else:
#    print('diff')

same
same
same
same
same
same
same
same
same
same
same
same
same
same
same
same
same
same
same
same
same
same
same
same


In [48]:
raw_data_df['lep_truthMatched'][2]==raw_data_df['lep_truthMatched'][1].all()

array([ True,  True])